# Hierarchical Clustering

In [ ]:
!pip install turicreate
!pip install numpy

In [ ]:
import os.path
from google.colab import drive

# mount Google Drive to /content/drive/My Drive/
if os.path.isdir("/content/drive/My Drive/"):
  print("Google Drive already mounted")
else:
  drive.mount('/content/drive')

Google Drive already mounted


In [ ]:
import os.path
import urllib.request
import tarfile
import zipfile
import gzip
from shutil import copy

def fetch_remote_datafile(filename, remote_url):
  if os.path.isfile("./" + filename):
    print("already have " + filename + " in workspace")
    return
  print("fetching " + filename + " from " + remote_url + "...")
  urllib.request.urlretrieve(remote_url, "./" + filename)

def cache_datafile_in_drive(filename):
  if os.path.isfile("./" + filename) == False:
    print("cannot cache " + filename + ", it is not in workspace")
    return
  
  data_drive_path = "/content/drive/My Drive/Colab Notebooks/"
  if os.path.isfile(data_drive_path + filename):
    print("" + filename + " has already been stored in Google Drive")
  else:
    print("copying " + filename + " to " + data_drive_path)
    copy("./" + filename, data_drive_path)
  

def load_datafile_from_drive(filename, remote_url=None):
  data_drive_path = "/content/drive/My Drive/Colab Notebooks/"
  if os.path.isfile("./" + filename):
    print("already have " + filename + " in workspace")
  elif os.path.isfile(data_drive_path + filename):
    print("have " + filename + " in Google Drive, copying to workspace...")
    copy(data_drive_path + filename, ".")
  elif remote_url != None:
    fetch_remote_datafile(filename, remote_url)
  else:
    print("error: you need to manually download " + filename + " and put in drive")
    
def extract_datafile(filename, expected_extract_artifact=None):
  if expected_extract_artifact != None and (os.path.isfile(expected_extract_artifact) or os.path.isdir(expected_extract_artifact)):
    
    print("files in " + filename + " have already been extracted")
  elif os.path.isfile("./" + filename) == False:
    print("error: cannot extract " + filename + ", it is not in the workspace")
  else:
    extension = filename.split('.')[-1]
    if extension == "zip":
      print("extracting " + filename + "...")
      data_file = open(filename, "rb")
      z = zipfile.ZipFile(data_file)
      for name in z.namelist():
          print("    extracting file", name)
          z.extract(name, "./")
      data_file.close()
    elif extension == "gz":
      print("extracting " + filename + "...")
      if filename.split('.')[-2] == "tar":
        tar = tarfile.open(filename)
        tar.extractall()
        tar.close()
      else:
        data_zip_file = gzip.GzipFile(filename, 'rb')
        data = data_zip_file.read()
        data_zip_file.close()
        extracted_file = open('.'.join(filename.split('.')[0:-1]), 'wb')
        extracted_file.write(data)
        extracted_file.close()
    elif extension == "tar":
      print("extracting " + filename + "...")
      tar = tarfile.open(filename)
      tar.extractall()
      tar.close()
    elif extension == "csv":
      print("do not need to extract csv")
    else:
      print("cannot extract " + filename)
      
def load_cache_extract_datafile(filename, expected_extract_artifact=None, remote_url=None):
  load_datafile_from_drive(filename, remote_url)
  extract_datafile(filename, expected_extract_artifact)
  cache_datafile_in_drive(filename)
  
load_cache_extract_datafile("people_wiki.sframe.zip", "people_wiki.sframe", "https://d3c33hcgiwev3.cloudfront.net/i5zBDt4bEemhxxJUtZcQoA_7fb43aab245846f09a8c48977883f0b6_people_wiki.sframe.zip?Expires=1590278400&Signature=Yl6JwzVhLnbYRI1PRicBdWKg~7UyAdfV3BoYSIJfGKdiBvndS~KargUUOi0sNKw1E52A5qr2nK~3~ZHJtuQe9TcE6O6pACHEBH0nyL4FG2m7nUo2NGIwtqReXYn0eO3wML2vSurqFUZE0qIRxIk7g5WvloHE~Ei0lBXc92K2ozU_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A")
load_cache_extract_datafile("em_utilities.py.zip", "em_utilities.py", "https://d3c33hcgiwev3.cloudfront.net/DCphrd4eEemzYQ60aJAwVA_fc87f6033fa34a2ab644e47ef86539cc_em_utilities.py.zip?Expires=1590364800&Signature=gb1PAlcZAY1zMykaXu5FCaUuVmmRaHDm53ARxr1cO8Nh7nVwC6YFHHXMeleqwiyo68hRwKeTwaAqWDHmlJ6VgYYLUc7us~i00CE5xYRmEMF0aWdQqcOzKgu1RKGKLl~wp~5e3Lj5MPKZrUv8iZWlE~102Yz3d5NlTW8J-~S8S7M_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A")


already have people_wiki.sframe.zip in workspace
files in people_wiki.sframe.zip have already been extracted
people_wiki.sframe.zip has already been stored in Google Drive
already have em_utilities.py.zip in workspace
files in em_utilities.py.zip have already been extracted
em_utilities.py.zip has already been stored in Google Drive


**Hierarchical clustering** refers to a class of clustering methods that seek to build a **hierarchy** of clusters, in which some clusters contain others. In this assignment, we will explore a top-down approach, recursively bipartitioning the data using k-means.

**Note to Amazon EC2 users**: To conserve memory, make sure to stop all the other notebooks before running this notebook.

## Import packages

In [ ]:
from __future__ import print_function # to conform python 2.x print to python 3.x
import turicreate
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import time
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
%matplotlib inline

## Load the Wikipedia dataset

In [ ]:
wiki = turicreate.SFrame('people_wiki.sframe/')

As we did in previous assignments, let's extract the TF-IDF features:

In [ ]:
wiki['tf_idf'] = turicreate.text_analytics.tf_idf(wiki['text'])

To run k-means on this dataset, we should convert the data matrix into a sparse matrix.

In [ ]:
from em_utilities import sframe_to_scipy # converter

# This will take about a minute or two.
wiki = wiki.add_row_number()
tf_idf, map_word_to_index = sframe_to_scipy(wiki, 'tf_idf')

To be consistent with the k-means assignment, let's normalize all vectors to have unit norm.

In [ ]:
from sklearn.preprocessing import normalize
tf_idf = normalize(tf_idf)

## Bipartition the Wikipedia dataset using k-means

Recall our workflow for clustering text data with k-means:

1. Load the dataframe containing a dataset, such as the Wikipedia text dataset.
2. Extract the data matrix from the dataframe.
3. Run k-means on the data matrix with some value of k.
4. Visualize the clustering results using the centroids, cluster assignments, and the original dataframe. We keep the original dataframe around because the data matrix does not keep auxiliary information (in the case of the text dataset, the title of each article).

Let us modify the workflow to perform bipartitioning:

1. Load the dataframe containing a dataset, such as the Wikipedia text dataset.
2. Extract the data matrix from the dataframe.
3. Run k-means on the data matrix with k=2.
4. Divide the data matrix into two parts using the cluster assignments.
5. Divide the dataframe into two parts, again using the cluster assignments. This step is necessary to allow for visualization.
6. Visualize the bipartition of data.

We'd like to be able to repeat Steps 3-6 multiple times to produce a **hierarchy** of clusters such as the following:
```
                      (root)
                         |
            +------------+-------------+
            |                          |
         Cluster                    Cluster
     +------+-----+             +------+-----+
     |            |             |            |
   Cluster     Cluster       Cluster      Cluster
```
Each **parent cluster** is bipartitioned to produce two **child clusters**. At the very top is the **root cluster**, which consists of the entire dataset.

Now we write a wrapper function to bipartition a given cluster using k-means. There are three variables that together comprise the cluster:

* `dataframe`: a subset of the original dataframe that correspond to member rows of the cluster
* `matrix`: same set of rows, stored in sparse matrix format
* `centroid`: the centroid of the cluster (not applicable for the root cluster)

Rather than passing around the three variables separately, we package them into a Python dictionary. The wrapper function takes a single dictionary (representing a parent cluster) and returns two dictionaries (representing the child clusters).

In [ ]:
def bipartition(cluster, maxiter=400, num_runs=4, seed=None):
    '''cluster: should be a dictionary containing the following keys
                * dataframe: original dataframe
                * matrix:    same data, in matrix format
                * centroid:  centroid for this particular cluster'''
    
    data_matrix = cluster['matrix']
    dataframe   = cluster['dataframe']
    
    # Run k-means on the data matrix with k=2. We use scikit-learn here to simplify workflow.
    kmeans_model = KMeans(n_clusters=2, max_iter=maxiter, n_init=num_runs, random_state=seed, n_jobs=1)
    kmeans_model.fit(data_matrix)
    centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_
    
    # Divide the data matrix into two parts using the cluster assignments.
    data_matrix_left_child, data_matrix_right_child = data_matrix[cluster_assignment==0], \
                                                      data_matrix[cluster_assignment==1]
    
    # Divide the dataframe into two parts, again using the cluster assignments.
    cluster_assignment_sa = turicreate.SArray(cluster_assignment) # minor format conversion
    dataframe_left_child, dataframe_right_child     = dataframe[cluster_assignment_sa==0], \
                                                      dataframe[cluster_assignment_sa==1]
        
    
    # Package relevant variables for the child clusters
    cluster_left_child  = {'matrix': data_matrix_left_child,
                           'dataframe': dataframe_left_child,
                           'centroid': centroids[0]}
    cluster_right_child = {'matrix': data_matrix_right_child,
                           'dataframe': dataframe_right_child,
                           'centroid': centroids[1]}
    
    return (cluster_left_child, cluster_right_child)

The following cell performs bipartitioning of the Wikipedia dataset. Allow 2+ minutes to finish.

Note. For the purpose of the assignment, we set an explicit seed (`seed=1`) to produce identical outputs for every run. In pratical applications, you might want to use different random seeds for all runs.

In [ ]:
%%time
wiki_data = {'matrix': tf_idf, 'dataframe': wiki} # no 'centroid' for the root cluster
left_child, right_child = bipartition(wiki_data, maxiter=100, num_runs=1, seed=0)

CPU times: user 3min 3s, sys: 2.17 s, total: 3min 5s
Wall time: 3min 2s


Let's examine the contents of one of the two clusters, which we call the `left_child`, referring to the tree visualization above.

In [ ]:
left_child

{'centroid': array([5.99333810e-05, 1.39284997e-05, 6.85264647e-05, ...,
        2.03016100e-06, 6.24672345e-06, 2.31038436e-06]), 'dataframe': Columns:
 	id	int
 	URI	str
 	name	str
 	text	str
 	tf_idf	dict
 
 Rows: Unknown
 
 Data:
 +----+-------------------------------+---------------------+
 | id |              URI              |         name        |
 +----+-------------------------------+---------------------+
 | 1  | <http://dbpedia.org/resour... |    Alfred J. Lewy   |
 | 2  | <http://dbpedia.org/resour... |    Harpdog Brown    |
 | 3  | <http://dbpedia.org/resour... | Franz Rottensteiner |
 | 4  | <http://dbpedia.org/resour... |        G-Enka       |
 | 5  | <http://dbpedia.org/resour... |    Sam Henderson    |
 | 6  | <http://dbpedia.org/resour... |    Aaron LaCrate    |
 | 7  | <http://dbpedia.org/resour... |   Trevor Ferguson   |
 | 8  | <http://dbpedia.org/resour... |     Grant Nelson    |
 | 9  | <http://dbpedia.org/resour... |     Cathy Caruth    |
 | 10 | <http://dbpedi

And here is the content of the other cluster we named `right_child`.

In [ ]:
right_child

{'centroid': array([0.00050277, 0.00038182, 0.00014018, ..., 0.        , 0.        ,
        0.        ]), 'dataframe': Columns:
 	id	int
 	URI	str
 	name	str
 	text	str
 	tf_idf	dict
 
 Rows: Unknown
 
 Data:
 +----+-------------------------------+-------------------------------+
 | id |              URI              |              name             |
 +----+-------------------------------+-------------------------------+
 | 0  | <http://dbpedia.org/resour... |         Digby Morrell         |
 | 17 | <http://dbpedia.org/resour... | Paddy Dunne (Gaelic footba... |
 | 21 | <http://dbpedia.org/resour... |         Ceiron Thomas         |
 | 22 | <http://dbpedia.org/resour... |          Adel Sellimi         |
 | 25 | <http://dbpedia.org/resour... |          Vic Stasiuk          |
 | 28 | <http://dbpedia.org/resour... |          Leon Hapgood         |
 | 30 | <http://dbpedia.org/resour... |           Dom Flora           |
 | 33 | <http://dbpedia.org/resour... |           Bob Reece           

## Visualize the bipartition

We provide you with a modified version of the visualization function from the k-means assignment. For each cluster, we print the top 5 words with highest TF-IDF weights in the centroid and display excerpts for the 8 nearest neighbors of the centroid.

In [ ]:
def display_single_tf_idf_cluster(cluster, map_index_to_word):
    '''map_index_to_word: SFrame specifying the mapping betweeen words and column indices'''
    
    wiki_subset   = cluster['dataframe']
    tf_idf_subset = cluster['matrix']
    centroid      = cluster['centroid']
    
    # Print top 5 words with largest TF-IDF weights in the cluster
    idx = centroid.argsort()[::-1]
    for i in range(5):
        print('{0}:{1:.3f}'.format(map_index_to_word['category'], centroid[idx[i]])),
    print('')
    
    # Compute distances from the centroid to all data points in the cluster.
    distances = pairwise_distances(tf_idf_subset, [centroid], metric='euclidean').flatten()
    # compute nearest neighbors of the centroid within the cluster.
    nearest_neighbors = distances.argsort()
    # For 8 nearest neighbors, print the title as well as first 180 characters of text.
    # Wrap the text at 80-character mark.
    for i in range(8):
        text = ' '.join(wiki_subset[nearest_neighbors[i]]['text'].split(None, 25)[0:25])
        print('* {0:50s} {1:.5f}\n  {2:s}\n  {3:s}'.format(wiki_subset[nearest_neighbors[i]]['name'],
              distances[nearest_neighbors[i]], text[:90], text[90:180] if len(text) > 90 else ''))
    print('')

Let's visualize the two child clusters:

In [ ]:
display_single_tf_idf_cluster(left_child, map_word_to_index)

113949:0.025
113949:0.017
113949:0.012
113949:0.011
113949:0.011

* Anita Kunz                                         0.97401
  anita e kunz oc born 1956 is a canadianborn artist and illustratorkunz has lived in london
   new york and toronto contributing to magazines and working
* Janet Jackson                                      0.97472
  janet damita jo jackson born may 16 1966 is an american singer songwriter and actress know
  n for a series of sonically innovative socially conscious and
* Madonna (entertainer)                              0.97475
  madonna louise ciccone tkoni born august 16 1958 is an american singer songwriter actress 
  and businesswoman she achieved popularity by pushing the boundaries of lyrical
* %C3%81ine Hyland                                   0.97536
  ine hyland ne donlon is emeritus professor of education and former vicepresident of univer
  sity college cork ireland she was born in 1942 in athboy co
* Jane Fonda                                     

In [ ]:
display_single_tf_idf_cluster(right_child, map_word_to_index)

113949:0.040
113949:0.036
113949:0.029
113949:0.029
113949:0.028

* Todd Williams                                      0.95468
  todd michael williams born february 13 1971 in syracuse new york is a former major league 
  baseball relief pitcher he attended east syracuseminoa high school
* Gord Sherven                                       0.95622
  gordon r sherven born august 21 1963 in gravelbourg saskatchewan and raised in mankota sas
  katchewan is a retired canadian professional ice hockey forward who played
* Justin Knoedler                                    0.95639
  justin joseph knoedler born july 17 1980 in springfield illinois is a former major league 
  baseball catcherknoedler was originally drafted by the st louis cardinals
* Chris Day                                          0.95648
  christopher nicholas chris day born 28 july 1975 is an english professional footballer who
   plays as a goalkeeper for stevenageday started his career at tottenham
* Tony Smith (football

The right cluster consists of athletes and artists (singers and actors/actresses), whereas the left cluster consists of non-athletes and non-artists. So far, we have a single-level hierarchy consisting of two clusters, as follows:

```
                                           Wikipedia
                                               +
                                               |
                    +--------------------------+--------------------+
                    |                                               |
                    +                                               +
         Non-athletes/artists                                Athletes/artists
```

Is this hierarchy good enough? **When building a hierarchy of clusters, we must keep our particular application in mind.** For instance, we might want to build a **directory** for Wikipedia articles. A good directory would let you quickly narrow down your search to a small set of related articles. The categories of athletes and non-athletes are too general to facilitate efficient search. For this reason, we decide to build another level into our hierarchy of clusters with the goal of getting more specific cluster structure at the lower level. To that end, we subdivide both the `athletes/artists` and `non-athletes/artists` clusters.

## Perform recursive bipartitioning

### Cluster of athletes and artists

To help identify the clusters we've built so far, let's give them easy-to-read aliases:

In [ ]:
non_athletes_artists   = left_child
athletes_artists       = right_child

Using the bipartition function, we produce two child clusters of the athlete cluster:

In [ ]:
# Bipartition the cluster of athletes and artists
left_child_athletes_artists, right_child_athletes_artists = bipartition(athletes_artists, maxiter=100, num_runs=6, seed=1)

The left child cluster mainly consists of athletes:

In [ ]:
display_single_tf_idf_cluster(left_child_athletes_artists, map_word_to_index)

113949:0.054
113949:0.043
113949:0.038
113949:0.035
113949:0.030

* Tony Smith (footballer, born 1957)                 0.94677
  anthony tony smith born 20 february 1957 is a former footballer who played as a central de
  fender in the football league in the 1970s and
* Justin Knoedler                                    0.94746
  justin joseph knoedler born july 17 1980 in springfield illinois is a former major league 
  baseball catcherknoedler was originally drafted by the st louis cardinals
* Chris Day                                          0.94849
  christopher nicholas chris day born 28 july 1975 is an english professional footballer who
   plays as a goalkeeper for stevenageday started his career at tottenham
* Todd Williams                                      0.94882
  todd michael williams born february 13 1971 in syracuse new york is a former major league 
  baseball relief pitcher he attended east syracuseminoa high school
* Todd Curley                                     

On the other hand, the right child cluster consists mainly of artists (singers and actors/actresses):

In [ ]:
display_single_tf_idf_cluster(right_child_athletes_artists, map_word_to_index)

113949:0.045
113949:0.043
113949:0.035
113949:0.031
113949:0.031

* Alessandra Aguilar                                 0.93880
  alessandra aguilar born 1 july 1978 in lugo is a spanish longdistance runner who specialis
  es in marathon running she represented her country in the event
* Heather Samuel                                     0.93999
  heather barbara samuel born 6 july 1970 is a retired sprinter from antigua and barbuda who
   specialized in the 100 and 200 metres in 1990
* Viola Kibiwot                                      0.94037
  viola jelagat kibiwot born december 22 1983 in keiyo district is a runner from kenya who s
  pecialises in the 1500 metres kibiwot won her first
* Ayelech Worku                                      0.94052
  ayelech worku born june 12 1979 is an ethiopian longdistance runner most known for winning
   two world championships bronze medals on the 5000 metres she
* Krisztina Papp                                     0.94105
  krisztina papp born 17

Our hierarchy of clusters now looks like this:
```
                                           Wikipedia
                                               +
                                               |
                    +--------------------------+--------------------+
                    |                                               |
                    +                                               +
         Non-athletes/artists                                Athletes/artists
                                                                    +
                                                                    |
                                                         +----------+----------+
                                                         |                     |
                                                         |                     |
                                                         +                     |
                                                     athletes               artists
```

Should we keep subdividing the clusters? If so, which cluster should we subdivide? To answer this question, we again think about our application. Since we organize our directory by topics, it would be nice to have topics that are about as coarse as each other. For instance, if one cluster is about baseball, we expect some other clusters about football, basketball, volleyball, and so forth. That is, **we would like to achieve similar level of granularity for all clusters.**

Both the athletes and artists node can be subdivided more, as each one can be divided into more descriptive professions (singer/actress/painter/director, or baseball/football/basketball, etc.). Let's explore subdividing the athletes cluster further to produce finer child clusters.

Let's give the clusters aliases as well:

In [ ]:
athletes    = left_child_athletes_artists
artists     = right_child_athletes_artists

### Cluster of athletes

In answering the following quiz question, take a look at the topics represented in the top documents (those closest to the centroid), as well as the list of words with highest TF-IDF weights.

Let us bipartition the cluster of athletes.

In [ ]:
left_child_athletes, right_child_athletes = bipartition(athletes, maxiter=100, num_runs=6, seed=1)

In [ ]:
display_single_tf_idf_cluster(left_child_athletes, map_word_to_index)
display_single_tf_idf_cluster(right_child_athletes, map_word_to_index)

113949:0.110
113949:0.102
113949:0.051
113949:0.046
113949:0.045

* Steve Springer                                     0.89370
  steven michael springer born february 11 1961 is an american former professional baseball 
  player who appeared in major league baseball as a third baseman and
* Dave Ford                                          0.89622
  david alan ford born december 29 1956 is a former major league baseball pitcher for the ba
  ltimore orioles born in cleveland ohio ford attended lincolnwest
* Todd Williams                                      0.89829
  todd michael williams born february 13 1971 in syracuse new york is a former major league 
  baseball relief pitcher he attended east syracuseminoa high school
* Justin Knoedler                                    0.90102
  justin joseph knoedler born july 17 1980 in springfield illinois is a former major league 
  baseball catcherknoedler was originally drafted by the st louis cardinals
* Kevin Nicholson (baseball)        

**Quiz Question**. Which diagram best describes the hierarchy right after splitting the `athletes` cluster? Refer to the quiz form for the diagrams.

**Caution**. The granularity criteria is an imperfect heuristic and must be taken with a grain of salt. It takes a lot of manual intervention to obtain a good hierarchy of clusters.

* **If a cluster is highly mixed, the top articles and words may not convey the full picture of the cluster.** Thus, we may be misled if we judge the purity of clusters solely by their top documents and words. 
* **Many interesting topics are hidden somewhere inside the clusters but do not appear in the visualization.** We may need to subdivide further to discover new topics. For instance, subdividing the `ice_hockey_football` cluster led to the appearance of runners and golfers.

### Cluster of non-athletes

Now let us subdivide the cluster of non-athletes.

In [ ]:
%%time 
# Bipartition the cluster of non-athletes
left_child_non_athletes_artists, right_child_non_athletes_artists = bipartition(non_athletes_artists, maxiter=100, num_runs=3, seed=1)

CPU times: user 10min 3s, sys: 8.65 s, total: 10min 12s
Wall time: 10min 1s


In [ ]:
display_single_tf_idf_cluster(left_child_non_athletes_artists, map_word_to_index)

113949:0.016
113949:0.013
113949:0.013
113949:0.012
113949:0.012

* Barry Sullivan (lawyer)                            0.97227
  barry sullivan is a chicago lawyer and as of july 1 2009 the cooney conway chair in advoca
  cy at loyola university chicago school of law
* Kayee Griffin                                      0.97444
  kayee frances griffin born 6 february 1950 is an australian politician and former australi
  an labor party member of the new south wales legislative council serving
* Christine Robertson                                0.97450
  christine mary robertson born 5 october 1948 is an australian politician and former austra
  lian labor party member of the new south wales legislative council serving
* James A. Joseph                                    0.97464
  james a joseph born 1935 is an american former diplomatjoseph is professor of the practice
   of public policy studies at duke university and founder of
* David Anderson (British Columbia politician)       0.9

In [ ]:
display_single_tf_idf_cluster(right_child_non_athletes_artists, map_word_to_index)

113949:0.039
113949:0.030
113949:0.023
113949:0.021
113949:0.015

* Madonna (entertainer)                              0.96092
  madonna louise ciccone tkoni born august 16 1958 is an american singer songwriter actress 
  and businesswoman she achieved popularity by pushing the boundaries of lyrical
* Janet Jackson                                      0.96153
  janet damita jo jackson born may 16 1966 is an american singer songwriter and actress know
  n for a series of sonically innovative socially conscious and
* Cher                                               0.96540
  cher r born cherilyn sarkisian may 20 1946 is an american singer actress and television ho
  st described as embodying female autonomy in a maledominated industry
* Laura Smith                                        0.96600
  laura smith is a canadian folk singersongwriter she is best known for her 1995 single shad
  e of your love one of the years biggest hits
* Natashia Williams                                  0

The clusters are not as clear, but the left cluster has a tendency to show important female figures, and the right one to show politicians and government officials.

Let's divide them further.

In [ ]:
female_figures = left_child_non_athletes_artists
politicians_etc = right_child_non_athletes_artists

**Quiz Question**. Let us bipartition the clusters `female_figures` and `politicians`. Which diagram best describes the resulting hierarchy of clusters for the non-athletes? Refer to the quiz for the diagrams.

**Note**. Use `maxiter=100, num_runs=6, seed=1` for consistency of output.

In [ ]:
left_child_female_figures, right_child_female_figures = bipartition(female_figures, maxiter=10, num_runs=6, seed=1)

In [ ]:
display_single_tf_idf_cluster(left_child_female_figures, map_word_to_index)

113949:0.037
113949:0.035
113949:0.032
113949:0.028
113949:0.025

* Kayee Griffin                                      0.95234
  kayee frances griffin born 6 february 1950 is an australian politician and former australi
  an labor party member of the new south wales legislative council serving
* Marcelle Mersereau                                 0.95551
  marcelle mersereau born february 14 1942 in pointeverte new brunswick is a canadian politi
  cian a civil servant for most of her career she also served
* Lucienne Robillard                                 0.95564
  lucienne robillard pc born june 16 1945 is a canadian politician and a member of the liber
  al party of canada she sat in the house
* Maureen Lyster                                     0.95691
  maureen anne lyster born 10 september 1943 is an australian politician she was an australi
  an labor party member of the victorian legislative assembly from 1985
* Liz Cunningham                                     0.95791
  eliz

In [ ]:
display_single_tf_idf_cluster(right_child_female_figures, map_word_to_index)

113949:0.018
113949:0.016
113949:0.013
113949:0.013
113949:0.010

* Lawrence W. Green                                  0.97499
  lawrence w green is best known by health education researchers as the originator of the pr
  ecede model and codeveloper of the precedeproceed model which has
* James A. Joseph                                    0.97535
  james a joseph born 1935 is an american former diplomatjoseph is professor of the practice
   of public policy studies at duke university and founder of
* Timothy Luke                                       0.97585
  timothy w luke is university distinguished professor of political science in the college o
  f liberal arts and human sciences as well as program chair of
* Archie Brown                                       0.97625
  archibald haworth brown cmg fba commonly known as archie brown born 10 may 1938 is a briti
  sh political scientist and historian in 2005 he became
* Jerry L. Martin                                    0.97689
  jerr

In [ ]:
left_child_politicians_etc, right_child_politicians_etc = bipartition(politicians_etc, maxiter=10, num_runs=6, seed=1)

In [ ]:
display_single_tf_idf_cluster(left_child_politicians_etc , map_word_to_index)

113949:0.124
113949:0.092
113949:0.015
113949:0.015
113949:0.014

* Janet Jackson                                      0.93383
  janet damita jo jackson born may 16 1966 is an american singer songwriter and actress know
  n for a series of sonically innovative socially conscious and
* Barbara Hershey                                    0.93509
  barbara hershey born barbara lynn herzstein february 5 1948 once known as barbara seagull 
  is an american actress in a career spanning nearly 50 years
* Lauren Royal                                       0.93717
  lauren royal born march 3 circa 1965 is a book writer from california royal has written bo
  th historic and novelistic booksa selfproclaimed angels baseball fan
* Alexandra Potter                                   0.93804
  alexandra potter born 1970 is a british author of romantic comediesborn in bradford yorksh
  ire england and educated at liverpool university gaining an honors degree in
* Cher                                    

In [ ]:
display_single_tf_idf_cluster(right_child_politicians_etc , map_word_to_index)

113949:0.027
113949:0.023
113949:0.017
113949:0.016
113949:0.014

* Julian Knowles                                     0.96905
  julian knowles is an australian composer and performer specialising in new and emerging te
  chnologies his creative work spans the fields of composition for theatre dance
* Peter Combe                                        0.97079
  peter combe born 20 october 1948 is an australian childrens entertainer and musicianmusica
  l genre childrens musiche has had 22 releases including seven gold albums two
* Craig Pruess                                       0.97120
  craig pruess born 1950 is an american composer musician arranger and gold platinum record 
  producer who has been living in britain since 1973 his career
* Ceiri Torjussen                                    0.97169
  ceiri torjussen born 1976 is a composer who has contributed music to dozens of film and te
  levision productions in the ushis music was described by
* Brenton Broadstock              